# Structured Output (Formato de Respuesta)

## ¿Por qué es bueno que nuestras Aplicaciones LLM y Agentes puedan devolver sus respuestas en salida estructurada?
* En lugar de devolver respuestas en lenguaje natural, a veces querremos que nuestras Aplicaciones LLM devuelvan sus respuestas en salida estructurada como objetos JSON, modelos Pydantic o dataclasses.

## ¿Cómo podemos conseguirlo en LangChain 1.0?
* La función `create_agent` de LangChain gestiona la salida estructurada automáticamente.
    * El usuario establece el esquema de salida estructurada deseado mediante el parámetro `response_format`,
    * El esquema que define el formato de salida estructurada soporta:
        * Modelos Pydantic: subclases de BaseModel con validación de campos
        * Dataclasses: dataclasses de Python con anotaciones de tipo
        * TypedDict: clases de diccionarios tipados
        * Esquema JSON: diccionario con especificación de esquema JSON
    * y cuando el modelo genera los datos estructurados, estos se capturan, validan y devuelven en la clave 'structured_response' del estado del agente.


```python
from pydantic import BaseModel, Field
from langchain.agents import create_agent


class ContactInfo(BaseModel):
    """Información de contacto de una persona."""
    name: str = Field(description="El nombre de la persona")
    email: str = Field(description="La dirección de correo electrónico de la persona")
    phone: str = Field(description="El número de teléfono de la persona")

agent = create_agent(
    model="gpt-4o-mini",
    response_format=ContactInfo
)

result = agent.invoke({
    "messages": [{
        "role": "user", 
        "content": "Extrae la información de contacto de: John Doe, john@example.com, (555) 123-4567"}]
})

print(result["structured_response"])
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')
```

* LangChain utiliza automáticamente ProviderStrategy cuando pasas un esquema y el Modelo soporta salida estructurada nativa, que es el caso más frecuente. Consulta la [documentación](https://docs.langchain.com/oss/python/langchain/structured-output) para ver otras opciones menos frecuentes.

## Ejemplo Básico

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from pydantic import BaseModel

class ArticleFormat(BaseModel):
    title: str
    subtitle: str
    body: str

agent = create_agent(
    model='gpt-4o-mini',
    system_prompt="Eres un periodista de investigación.",
    response_format=ArticleFormat
)

question = HumanMessage(content="Escribe un artículo breve explicando brevemente las principales teorías de la conspiración sobre quién mató a JFK")

response = agent.invoke(
    {"messages": [question]}
)

article = response["structured_response"]
article_title = article.title
article_subtitle = article.subtitle
article_body = article.body

print(f"El periodista escribió un artículo llamado {article_title}")
print(f"El artículo trataba sobre {article_subtitle}")
print(f"Este es el cuerpo del artículo:\n {article_body}")

## Expliquemos el código anterior en términos sencillos

A continuación se presenta una **explicación simple, apta para principiantes, línea por línea** de lo que hace este código de LangChain 1.0. Explicaremos cada parte en lenguaje claro.

---

#### 1. Importando las herramientas que necesitamos

```python
from langchain.agents import create_agent
```

* Esto importa una función auxiliar llamada `create_agent`.
* Un **agente** es un asistente de IA que puede recibir instrucciones y generar respuestas.
* Piensa en ello como "crear un trabajador de IA".

---

```python
from langchain.messages import HumanMessage
```

* Esto importa `HumanMessage`, que representa **algo que un humano dice a la IA**.
* LangChain utiliza objetos de mensaje en lugar de cadenas de texto para mantener las conversaciones estructuradas.

---

```python
from pydantic import BaseModel
```

* Esto importa `BaseModel` de **Pydantic**.
* Pydantic se utiliza para definir **datos estructurados** (datos con campos fijos).
* Lo usaremos para decirle a la IA **exactamente qué formato debe tener su respuesta**.

---

#### 2. Definiendo la estructura de la respuesta

```python
class ArticleFormat(BaseModel):
```

* Esto define un **modelo de datos** llamado `ArticleFormat`.
* Describe cómo debería ser una respuesta válida de la IA.

---

```python
    title: str
    subtitle: str
    body: str
```

* Estos son los campos requeridos en la respuesta:

  * `title`: una cadena de texto
  * `subtitle`: una cadena de texto
  * `body`: una cadena de texto
* La IA **debe** devolver los tres, o la respuesta fallará en la validación.

👉 En términos simples:

> "La IA debe responder en forma de un artículo con un título, subtítulo y cuerpo."

---

#### 3. Creando el agente de IA

```python
agent = create_agent(
```

* Esto crea la instancia del agente de IA.
* De ahora en adelante, `agent` es tu periodista de IA.

---

```python
    model='gpt-4o-mini',
```

* Esto le dice a LangChain qué modelo de IA usar.
* `gpt-4o-mini` es un modelo de lenguaje ligero y rápido.

---

```python
    system_prompt="Eres un periodista de investigación.",
```

* Esta es una **instrucción de sistema**.
* Establece el rol y comportamiento de la IA.
* La IA intentará responder **como un periodista**, no como un chatbot.

---

```python
    response_format=ArticleFormat
)
```

* Esto fuerza a que la respuesta de la IA coincida con el esquema `ArticleFormat`.
* LangChain analizará automáticamente la salida en datos estructurados.

👉 Esto es potente porque:

* Sin conjeturas
* Sin análisis de texto
* Sin manejo complicado de JSON

---

#### 4. Creando la pregunta del usuario

```python
question = HumanMessage(
    content="Escribe un artículo breve explicando brevemente las principales teorías de la conspiración sobre quién mató a JFK"
)
```

* Esto crea un **mensaje humano**.
* `content` es lo que el usuario le está preguntando a la IA.
* Envolverlo en `HumanMessage` lo hace compatible con el sistema de mensajes de LangChain.

---

#### 5. Enviando el mensaje al agente

```python
response = agent.invoke(
    {"messages": [question]}
)
```

* Esto envía el mensaje al agente de IA.
* `invoke()` ejecuta el agente y espera la respuesta.
* La entrada es un diccionario que contiene una lista de mensajes.

👉 Incluso para una pregunta, LangChain espera una **lista de mensajes**.

---

#### 6. Extrayendo la respuesta estructurada

```python
article = response["structured_response"]
```

* La salida del agente contiene una respuesta analizada.
* Como usamos `response_format=ArticleFormat`, LangChain ya convirtió la salida de la IA en un objeto `ArticleFormat`.

👉 En este punto:

* `article` **no es texto**
* Es un objeto Python con campos: `title`, `subtitle` y `body`

---

#### 7. Accediendo a cada campo del artículo

```python
article_title = article.title
article_subtitle = article.subtitle
article_body = article.body
```

* Estas líneas extraen cada parte del artículo.
* Esto funciona igual que acceder a atributos en cualquier objeto Python.

---

#### 8. Imprimiendo los resultados

```python
print(f"El periodista escribió un artículo llamado {article_title}")
```

* Imprime el título del artículo.

---

```python
print(f"El artículo trataba sobre {article_subtitle}")
```

* Imprime el subtítulo (de qué trata el artículo).

---

```python
print(f"Este es el cuerpo del artículo:\n {article_body}")
```

* Imprime el cuerpo del artículo.
* `\n` añade una nueva línea antes del texto.

---

#### 🧠 Resumen general

Este código:

1. Define un **formato de artículo estructurado**
2. Crea un **periodista de IA**
3. Le hace una pregunta
4. Fuerza a la IA a responder en una **estructura limpia y predecible**
5. Extrae e imprime cada parte del artículo

---

#### 🚀 Por qué este enfoque es potente

* Sin análisis complicado de cadenas
* Tipado fuerte y validación

## Principales opciones para formatear la respuesta de la aplicación en LangChain
* Pydantic y Dataclass son las formas más frecuentes de establecer la estructura de la salida en LangChain. Consulta información más detallada y ejemplos sobre estas y otras opciones en esta [Página de Documentación de LangChain](https://docs.langchain.com/oss/python/langchain/structured-output).

## Cómo ejecutar este código desde Visual Studio Code
* Abre el Terminal.
* Asegúrate de estar en la carpeta del proyecto.
* Asegúrate de tener el entorno poetry activado.
* Introduce y ejecuta el siguiente comando:
    * `python 005-structured-output.py`